<h4>Background</h4>
<h5>Squared exponential kernel</h5>
<p>See the squared exponential kernel formula in the Rasmussen &amp; Williams (R&amp;W) textbook Eq. 4.9:</p>
<div class="math">$$
k_{\text{sq-exp}}(x, x') = \exp \left( -\frac{(x - x')^2}{2 \ell^2} \right)
$$</div>
<p>Hyperparameters: <span class="math">\(\ell &gt; 0\)</span></p>
<h5>Matern kernel</h5>
<p>See the Matern kernel formula in the R&amp;W textbook Eq. 4.14: </p>
<div class="math">$$
k_{\text{Matern}}(x, x') = 
    \frac{2^{1-\nu}}{\Gamma(\nu)}
    \bigg(
    \frac
        {\sqrt{2\nu} |x - x'| }
        { \ell }
    \bigg)^\nu
    K^{\text{bessel}}_\nu\bigg(\frac{\sqrt{2\nu} |x - x'| }{\ell}\bigg)
$$</div>
<p>Hyperparameters: <span class="math">\(\ell &gt; 0\)</span> and <span class="math">\(\nu &gt; 0\)</span></p>
<p>Useful:</p>
<ul>
<li><span class="math">\(\Gamma(\cdot)\)</span> is the Gamma function, available in SciPy as <a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.gamma.html">from scipy.special import gamma</a></li>
<li><span class="math">\(K^{\text{bessel}}_{\nu}(\cdot)\)</span> is the modified Bessel function, available in SciPy as <a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.kv.html">from scipy.special import kv</a></li>
</ul>
<h5>Helpful Hint</h5>
<p>You might wish to define two Python functions, <code>sqexp_kernel_func</code> and <code>matern_kernel_func</code> to compute these kernels given any two possible inputs, <span class="math">\(x\)</span> and <span class="math">\(x'\)</span>. It is probably smart to write these functions in a vectorized form, so that given two vectors of length <span class="math">\(A\)</span> and <span class="math">\(B\)</span>, the function returns a kernel matrix of size <span class="math">\(A x B\)</span>.</p>
<h2><a name="problem-1">Problem 1: Sampling from the Prior</a></h2>
<p>Write Python code to sample function values from a Gaussian Process (GP) prior.</p>
<p>You should sample the function values that correspond to a set of at least 200 evenly-spaced test points <span class="math">\(\{x_i\}\)</span> between -20 and 20. One way to generate a 1D array of <span class="math">\(G\)</span> points would be: <code>x_grid_G = np.linspace(-20, 20, G)</code>. </p>
<p>Your prior should be specified by:</p>
<ul>
<li>mean function equal to 0 </li>
<li>covariance function ("kernel") which is specified as a Python function, which you can <a href="https://stackoverflow.com/questions/706721/how-do-i-pass-a-method-as-a-parameter-in-python">pass as the named "cov_func" argument below</a> </li>
</ul>
<p>Here's an example template (you don't have to use it):</p>

In [1]:
def draw_GP_prior_samples_at_x_grid(
        x_grid_G, mean_func, cov_func,
        random_seed=42,
        n_samples=1):
    """ Draw sample from GP prior given mean/cov functions

    Args
    ----
    x_grid_G : 1D array, size n_grid_pts (G)
        Contains G specific x_i values to evaluate function at
    mean_func : function, maps (1D arr size A) to (1D arr size A)
        Computes mean value $m(x_i)$ at each input x_i value 
    cov_func : function, maps (1D arr size A, 1D arr size B) to (2D AxB)
        Computes covariance (kernel) value at each pair of inputs.
    random_seed : int
        See for the random number generator
    n_samples : int
        Number of samples to draw from the prior

    Returns
    -------
    f_SG : 2D array, n_samples (S) x n_grid_pts (G)
        Contains sampled function values at each point of x_grid
    """
    # TODO compute mean at each grid point

    # TODO compute covariance matrix across grid points

    # Use consistent random number generator for reproducibility
    prng = np.random.RandomState(int(random_seed))
    TODO = prng.multivariate_normal(TODO)

    return TODO

<p>To demonstrate your implementation, you'll make plots of sampled function values. Each individual plot should show a line plot of the test grid points <span class="math">\(x_i\)</span> and the corresponding sampled function values <span class="math">\(f_i = f(x_i)\)</span>. Use a matplotlib line style '.-' to show both the specific <span class="math">\(\{x_i, f_i\}\)</span> pair values and connecting lines.</p>
<p>For Problem 1, your report PDF should include:</p>
<p>a. 1 row x 3 column grid of plots, where each panel shows 5 samples from the prior using a squared exponential kernel</p>
<ul>
<li>For the columns, try 3 possible length scale values for <span class="math">\(\ell\)</span>: 0.25, 1.0, and 4.0</li>
</ul>
<p>b. Short text description of what each of the SE kernel's hyperparameters do (in terms of qualitative trends). A few short but complete sentences.</p>
<p>c. 3 row x 3 column grid of plots, where each panel shows 5 samples from the prior using a Matern kernel</p>
<ul>
<li>For the columns, try 3 possible lengthscales for <span class="math">\(\ell\)</span>: 0.25, 1.0, 4.0</li>
<li>For the rows, try 3 possible <span class="math">\(\nu\)</span> values: 0.5, 2, 8</li>
</ul>
<p>d. Short text description of what each of the Matern kernel's hyperparameters do (in terms of qualitative trends). A few short but complete sentences.</p>
<h2><a name="problem-2">Problem 2: Sampling from the Posterior</a></h2>
<p>Consider the following training data with <span class="math">\(N=6\)</span> example pairs of <span class="math">\(x\)</span> and <span class="math">\(y\)</span> values:</p>
<div class="highlight"><pre><span></span>x_train_N = np.asarray([-2.,    -1.8,   -1.,  1.,  1.8,     2.])
y_train_N = np.asarray([-3.,  0.2224,    3.,  3.,  0.2224, -3.])
</pre></div>


<p>As in R&amp;W Ch. 2's section on 'Prediction using Noisy Observations', we assume that the observed <span class="math">\(y_i\)</span> values are themselves 'noisy' perturbations of the true function values <span class="math">\(f(x_i)\)</span> that are modeled by a GP. So the likelihood is: <span class="math">\(y_i | x_i \sim \mathcal{N}(f(x_i), \sigma^2)\)</span>.
For this problem, you can assume the likelihood noise standard deviation is <span class="math">\(\sigma = 0.1\)</span>.</p>
<p>Write Python code to sample the function values <span class="math">\(\{f_i\}_{i=1}^G\)</span> that correspond to a given grid of new <span class="math">\(\{x^{\text{grid}}_i\}_{i=1}^G\)</span> values from a Gaussian Process (GP) <strong>posterior</strong> given this training data. Use the same GP prior specification as in Problem 1 (same mean function, same possible kernel functions).</p>
<p>Use a one-dimensional grid of test x points between -20 and 20, with at least 200 grid points.</p>

In [2]:
def draw_GP_posterior_samples_at_x_grid(
        x_train_N, y_train_N, x_grid_G, mean_func, cov_func,
        sigma=0.1,
        random_seed=42,
        n_samples=1):
    """ Draw sample from GP posterior given training data and mean/cov

    Args
    ----
    x_train_N : 1D array, size n_train_examples (N)
        Each entry i provides the x value observed at training example i 
    y_train_N : 1D array, size n_train_examples (N)
        Each entry i provides the y value observed at training example i 
    sigma : scalar float
        Specifies the standard deviation of the likelihood.
        y_i drawn from a 1D Normal with mean f(x_i) and std. dev. \sigma.
    Other args same as earlier function: draw_GP_prior_samples_at_x_grid

    Returns
    -------
    f_SG : 2D array, n_samples (S) x n_grid_pts (G)
        Contains sampled function values at each point of x_grid
    """
    TODO

<p>For Problem 2, your report PDF should include:</p>
<p>a. 1 row x 3 col grid of plots, where each panel shows 5 posterior samples using a squared exponential kernel</p>
<ul>
<li>For the columns, try 3 possible length scale values for <span class="math">\(\ell\)</span>: 0.25, 1.0, and 4.0</li>
</ul>
<p>b. Short answer (1-3 sentences): What happens at <span class="math">\(x=0\)</span> that differs across the three different choices of <span class="math">\(L\)</span>? Why? </p>
<p>c. 3 row x 3 col grid of plots, where each panel shows 5 posterior samples using a Matern kernel</p>
<ul>
<li>For the columns, try 3 possible length scale values for <span class="math">\(\ell\)</span>: 0.25, 1.0, and 4.0</li>
<li>For the rows, try 3 possible <span class="math">\(\nu\)</span> values: 0.5, 2, 8</li>
</ul>
<p>d. Short answer (a few sentences): Consider the plot with L=4 and nu=8. What (roughly) is a typical value of a posterior sample around <span class="math">\(x=-18\)</span> or <span class="math">\(x=+18\)</span>? Why? What do you expect the typical value would be at <span class="math">\(x=+100\)</span>? If you wanted this typical value to be something else, how would you achieve that?</p>
<p>e. Short answer (a few sentences): Consider the plot with L=4 and nu=8. What (roughly) is the <em>standard deviation</em> of a posterior sample around <span class="math">\(x=-18\)</span> or <span class="math">\(x=+18\)</span>? Why? What do you expect the standard deviation of posterior samples would be at <span class="math">\(x=+100\)</span>? How would you set the standard deviation to a specific desired value (like 42.0)?</p>
<h2><a name="coding-clarifications">Coding Clarifications</a></h2>
<p>For both problems, your implementation should of course not use any third-party Gaussian Process libraries, but instead be written from scratch to turn the math/pseudocode provided in resources like R&amp;W Ch. 2 into useful Python code. Any questions should be posted to the HW1 discussion on Canvas.</p>
<p>You may use existing libraries (e.g. numpy, scipy) for basic functions like:</p>
<ul>
<li>random number generation</li>
<li>array/matrix operations (product, inverse, linear solve, cholesky, eigenvalues)</li>
<li>special functions (like bessel or gamma)</li>
</ul>
<p>Be sure to turn in your complete code as an appendix. This could be just a concatenation of one-or-more "<em>.py" files, or a "</em>.ipynb" notebook if you like that format. Just make sure it is somewhat easy to follow (label the parts, use clear variable names, use decent doc-strings).</p>
<p>We will be unlikely to run your code, but we will possibly <em>read</em> it and thus it must clearly solve the problem at hand.</p>
<p>We reserve the right to have you reproduce key parts of your code at a whiteboard without references if we have questions about integrity. You should be able to explain any code you turn in (because you must have developed it yourself).</p>